In [ ]:
%reset -f

%run REV.ipynb

In [ ]:
from preprocessing import *

In [ ]:
df_TRAS = pd.read_excel('TRAS_BACKUP.xlsx', sheet_name='Sheet1')
TRAS=df_TRAS.values
print(TRAS.shape)


In [ ]:
#######################################################################################

theta_inf=300; h_conv=10; # W/m2K 

# PA12 (Nylon12) specs
K=0.22; # W/mK
roh0=1100; # kg/m3 
C=1590; # J/kgK
######################

theta_0=300; # theta_0 : Initial condition for ALL nodes

n2=N_Material_grid
n3=4 # z direction

Scan_speed = 0.01 # m/s

w=Laser_Diameter/2 # laser radius
Power=200; #Watt
I_0 = 2*Power/np.pi/w/w/L; # divided by L, in order to fit the dimension
beta=80; #Optical extinction coefficient


T_step=dL/Scan_speed # time which is taken to go through one material grid interval  
T_total = T_step*3 * (len(Total_points)-1) + T_step # total time (including cooling time at the last)
# 3: Because we already configured laser-material grid system like this

step_n=25 # how many time steps (dt) in one material grid interval
dt=T_step/step_n

times=np.linspace(0,T_total, step_n*3*(len(Total_points)-1)+step_n + 1)

k1=K*dt/roh0/C/dL/dL
k2=dt/roh0/C

########################################################################################

In [ ]:
GRADIENTS=np.zeros(TRAS.shape[0])

start=time.time()

for i in range(0,TRAS.shape[0]):

    print(i)

    PATH=TRAS[i]


    THETA=np.zeros((len(times),n2,n2,n3))
    THETA[0,:,:,:]=theta_0; 
    
    # Interpolating from laser grid path to material grid path
    PATH_converted=N_Material_grid * ( 3*(PATH//N_Laser_grid)+2 ) + ( 3*(PATH%N_Laser_grid)+2 )
    p_interval=np.int32((PATH_converted[1:]-PATH_converted[:-1])/3)
    PATH_NEW=np.array([ PATH_converted[:-1], PATH_converted[:-1]
                       +p_interval, PATH_converted[:-1]+2*p_interval ]).T.reshape(-1)
    PATH_NEW=np.append(PATH_NEW,PATH_converted[-1])

    count=0
    for LASER_POINTER in PATH_NEW:

        Laser=np.zeros((n2,n2,n3))

        laser_center_x=LASER_POINTER%N_Material_grid
        laser_center_y=LASER_POINTER//N_Material_grid
        
        Laser[laser_center_x, laser_center_y, 0] = k2*I_0*np.exp(  -beta * dL*0)
        tmp2= Laser[laser_center_x, laser_center_y, 0] * np.exp ( -2 * dL**2/w**2)
        tmp3= Laser[laser_center_x, laser_center_y, 0] * np.exp ( -4 * dL**2/w**2)
        Laser[laser_center_x-1, laser_center_y, 0] = tmp2
        Laser[laser_center_x+1, laser_center_y, 0] = tmp2
        Laser[laser_center_x, laser_center_y-1, 0] = tmp2
        Laser[laser_center_x, laser_center_y+1, 0] = tmp2
        Laser[laser_center_x-1, laser_center_y-1, 0] = tmp3
        Laser[laser_center_x+1, laser_center_y+1, 0] = tmp3
        Laser[laser_center_x-1, laser_center_y+1, 0] = tmp3
        Laser[laser_center_x+1, laser_center_y-1, 0] = tmp3
        
        for z_index in range(0,n3-2):
            Laser[laser_center_x-1:laser_center_x+2, laser_center_y-1:laser_center_y+2, z_index+1]= \
            Laser[laser_center_x-1:laser_center_x+2, laser_center_y-1:laser_center_y+2, z_index] \
            * np.exp(  -beta * dL)
        
        

        for t in range(count*step_n,(count+1)*step_n):

            THETA[t+1,1:n2-1,1:n2-1,1:n3-1]=THETA[t,1:n2-1,1:n2-1,1:n3-1] \
            +k1*(THETA[t,2:n2,1:n2-1,1:n3-1]+THETA[t,0:n2-2,1:n2-1,1:n3-1] \
                 +THETA[t,1:n2-1,2:n2,1:n3-1]+THETA[t,1:n2-1,0:n2-2,1:n3-1] \
                 +THETA[t,1:n2-1,1:n2-1,2:n3]+THETA[t,1:n2-1,1:n2-1,0:n3-2] \
                 -6*THETA[t,1:n2-1,1:n2-1,1:n3-1])+Laser[1:n2-1,1:n2-1,1:n3-1]

            # update boundary conditions
            THETA[t+1,:,:,0]=( K/(h_conv*dL+K) ) * THETA[t,:,:,1] + h_conv*dL*theta_inf/(h_conv*dL+K)  # convection
            
            # adiabatic conditions
            THETA[t+1,0,0:n2-1,1:n3-1]=THETA[t,1,0:n2-1,1:n3-1]
            THETA[t+1,n2-1,1:n2,1:n3-1]=THETA[t,n2-2,1:n2,1:n3-1]
            THETA[t+1,1:n2,0,1:n3-1]=THETA[t,1:n2,1,1:n3-1]
            THETA[t+1,0:n2-1,n2-1,1:n3-1]= THETA[t,0:n2-1,n2-2,1:n3-1]
            THETA[t+1,:,:,n3-1]=THETA[t,:,:,n3-2]
            

        count=count+1


    Tfinal=len(times)-1

    grad_x = (THETA[Tfinal,x_arr+1,y_arr,1:n3-1]-THETA[Tfinal,x_arr-1,y_arr,1:n3-1])/2/dL
    grad_y = (THETA[Tfinal,x_arr,y_arr+1,1:n3-1]-THETA[Tfinal,x_arr,y_arr-1,1:n3-1])/2/dL
    grad_z = (THETA[Tfinal,x_arr,y_arr,2:n3]-THETA[Tfinal,x_arr,y_arr,0:n3-2])/2/dL

    gradient_avg=np.mean(np.sqrt(grad_x**2+grad_y**2+grad_z**2))
    GRADIENTS[i]=gradient_avg



end=time.time()
elapsed=end-start
print("elapsed time is : ", elapsed)
# GRADIENTS.argmin()

GRADIENTS_sorted=np.sort(GRADIENTS)
TRAS_sorted=TRAS[np.argsort(GRADIENTS),:]

In [ ]:
# labeling for paths
LABEL_validpaths=np.zeros(GRADIENTS_sorted.shape[0]).astype(np.int32)
medd1=np.median(GRADIENTS_sorted, axis=0)
count=0
for i in range(0,GRADIENTS_sorted.shape[0]):
    if GRADIENTS_sorted[i]<=medd1:
        LABEL_validpaths[i]=1
        count+=1

In [ ]:
grad=np.reshape(GRADIENTS_sorted, (-1, 1))
lbl=np.reshape(LABEL_validpaths, (-1, 1))

df = pd.DataFrame (np.hstack((TRAS_sorted,grad,lbl)))
filepath = 'paths_data.xlsx'
df.to_excel(filepath, index=False)

In [ ]:
############################## RECORDER ############################## 

import pandas as pd
df_valid = pd.read_excel('paths_data.xlsx', sheet_name='Sheet1')

In [ ]:
nn=273

PATH=df_valid.T[nn][:-2].astype(np.int32).values

movie_name="movie_" + str(nn) + ".mp4"

import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np


THETA=np.zeros((len(times),n2,n2,n3))
THETA[0,:,:,:]=theta_0; 

############## plotting ###############
fig = plt.figure()
ims = [] # for video saving
z=THETA[0,:,:,0]
im=plt.imshow(z.T, extent=[0,L,L,0],interpolation = 'bilinear', animated=True); 

plt.title("Image plot of temperature");
ims.append([im])
#######################################

# Interpolating from laser grid path to material grid path
PATH_converted=N_Material_grid * ( 3*(PATH//N_Laser_grid)+2 ) + ( 3*(PATH%N_Laser_grid)+2 )
p_interval=np.int32((PATH_converted[1:]-PATH_converted[:-1])/3)
PATH_NEW=np.array([ PATH_converted[:-1], PATH_converted[:-1]
                   +p_interval, PATH_converted[:-1]+2*p_interval ]).T.reshape(-1)
PATH_NEW=np.append(PATH_NEW,PATH_converted[-1])


count=0
for LASER_POINTER in PATH_NEW:

    Laser=np.zeros((n2,n2,n3))

    laser_center_x=LASER_POINTER%N_Material_grid
    laser_center_y=LASER_POINTER//N_Material_grid

    Laser[laser_center_x, laser_center_y, 0] = k2*I_0*np.exp(  -beta * dL*0)
    tmp2= Laser[laser_center_x, laser_center_y, 0] * np.exp ( -2 * dL**2/w**2)
    tmp3= Laser[laser_center_x, laser_center_y, 0] * np.exp ( -4 * dL**2/w**2)
    Laser[laser_center_x-1, laser_center_y, 0] = tmp2
    Laser[laser_center_x+1, laser_center_y, 0] = tmp2
    Laser[laser_center_x, laser_center_y-1, 0] = tmp2
    Laser[laser_center_x, laser_center_y+1, 0] = tmp2
    Laser[laser_center_x-1, laser_center_y-1, 0] = tmp3
    Laser[laser_center_x+1, laser_center_y+1, 0] = tmp3
    Laser[laser_center_x-1, laser_center_y+1, 0] = tmp3
    Laser[laser_center_x+1, laser_center_y-1, 0] = tmp3

    for z_index in range(0,n3-2):
        Laser[laser_center_x-1:laser_center_x+2, laser_center_y-1:laser_center_y+2, z_index+1]= \
        Laser[laser_center_x-1:laser_center_x+2, laser_center_y-1:laser_center_y+2, z_index] \
        * np.exp(  -beta * dL)



    for t in range(count*step_n,(count+1)*step_n):

        THETA[t+1,1:n2-1,1:n2-1,1:n3-1]=THETA[t,1:n2-1,1:n2-1,1:n3-1] \
        +k1*(THETA[t,2:n2,1:n2-1,1:n3-1]+THETA[t,0:n2-2,1:n2-1,1:n3-1] \
             +THETA[t,1:n2-1,2:n2,1:n3-1]+THETA[t,1:n2-1,0:n2-2,1:n3-1] \
             +THETA[t,1:n2-1,1:n2-1,2:n3]+THETA[t,1:n2-1,1:n2-1,0:n3-2] \
             -6*THETA[t,1:n2-1,1:n2-1,1:n3-1])+Laser[1:n2-1,1:n2-1,1:n3-1]
        
        # update boundary conditions
        THETA[t+1,:,:,0]=( K/(h_conv*dL+K) ) * THETA[t,:,:,1] + h_conv*dL*theta_inf/(h_conv*dL+K)  # convection

        # adiabatic conditions
        THETA[t+1,0,0:n2-1,1:n3-1]=THETA[t,1,0:n2-1,1:n3-1]
        THETA[t+1,n2-1,1:n2,1:n3-1]=THETA[t,n2-2,1:n2,1:n3-1]
        THETA[t+1,1:n2,0,1:n3-1]=THETA[t,1:n2,1,1:n3-1]
        THETA[t+1,0:n2-1,n2-1,1:n3-1]= THETA[t,0:n2-1,n2-2,1:n3-1]
        THETA[t+1,:,:,n3-1]=THETA[t,:,:,n3-2]
        
    
    ############## plotting ###############
    z=THETA[(count+1)*step_n,1:n2-1,1:n2-1,0]
    im=plt.imshow(z.T, extent=[0,L,L,0],interpolation = 'bilinear', animated=True); 
    plt.title("Image plot of temperature");
    ims.append([im])
    ######################################\

    count=count+1

    
############# plotting #############
plt.colorbar(im)
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                repeat_delay=1000)

ani.save(movie_name, writer="ffmpeg")
plt.show()
######################################



In [ ]:
im=plt.imshow(z.T, extent=[0,L,L,0],interpolation = 'bilinear', animated=True)
plt.colorbar(im)
plt.title("Image plot of temperature", fontsize=12)
ax = plt.gca()
ax.set_xlabel('x (m)', fontsize=10)
ax.set_ylabel('y (m)', fontsize=10)

shot_name="temp_plot_" + str(nn) + ".eps"

plt.savefig(shot_name, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
with open("time_elapse_brute_force.txt","a") as f:
    f.write("\n----------\n")
    f.write("Elapsed time for step {}:".format(step_n))
    f.write('{}(s)'.format(elapsed))
    f.write("\n----------\n")